# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
clean_data = pd.read_csv("../output_data/Blaylock_cities.csv")
clean_data.head()

,Unnamed: 0,city,city_lat,city_lon,city_temp,city_hum,city_cloud,city_wind,city_country,city_date
0,0,waddan,29.16,16.14,61.41,64,100,7.18,LY,1604090542
1,1,ushuaia,-54.80,-68.30,48.20,76,75,24.16,AR,1604090386
2,2,aden,12.78,45.04,82.44,79,0,14.74,YE,1604090424
3,3,chuy,-33.70,-53.46,57.70,77,62,8.90,UY,1604090543
4,4,saint-philippe,-21.36,55.77,64.40,77,90,9.17,RE,1604090545


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
# Store latitude and longitude in locations and humidity as the weight
locations = clean_data[["city_lat", "city_lon"]]
humidity = clean_data["city_hum"]

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
clean_data.head()

,Unnamed: 0,city,city_lat,city_lon,city_temp,city_hum,city_cloud,city_wind,city_country,city_date
0,0,waddan,29.16,16.14,61.41,64,100,7.18,LY,1604090542
1,1,ushuaia,-54.80,-68.30,48.20,76,75,24.16,AR,1604090386
2,2,aden,12.78,45.04,82.44,79,0,14.74,YE,1604090424
3,3,chuy,-33.70,-53.46,57.70,77,62,8.90,UY,1604090543
4,4,saint-philippe,-21.36,55.77,64.40,77,90,9.17,RE,1604090545


In [8]:
# Narrow down the DataFrame to find your ideal weather condition. For example:

#   * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

#   * **Note:** Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.
ideal_weather = clean_data.loc[((clean_data['city_temp'] < 80) & (clean_data['city_temp'] > 70)) &
                               (clean_data['city_wind'] < 10) &
                               (clean_data['city_cloud'] == 0)].dropna()

ideal_weather

,Unnamed: 0,city,city_lat,city_lon,city_temp,city_hum,city_cloud,city_wind,city_country,city_date
5,5,rikitea,-23.12,-134.97,75.99,68,0,7.94,PF,1604090342
63,63,ovalle,-30.60,-71.20,77.92,31,0,6.91,CL,1604090817
77,77,aswan,24.09,32.91,78.64,35,0,5.55,EG,1604090822
118,118,agadez,19.75,10.25,72.55,20,0,9.66,NE,1604090838
222,222,mecca,21.43,39.83,77.29,63,0,3.44,SA,1604090877
223,223,hays,30.05,-98.03,71.01,30,0,1.99,US,1604090877
234,234,salalah,17.02,54.09,77.00,69,0,5.82,OM,1604090883
237,237,san carlos de bariloche,-41.15,-71.31,75.20,8,0,8.05,AR,1604090885
264,264,khipro,25.83,69.38,73.69,20,0,3.94,PK,1604090895
266,266,siguiri,11.67,-9.50,76.42,33,0,1.34,GN,1604090896


In [9]:
ideal_weather.count()

Unnamed: 0      20
city            20
city_lat        20
city_lon        20
city_temp       20
city_hum        20
city_cloud      20
city_wind       20
city_country    20
city_date       20
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = ideal_weather[['city','city_country','city_lat','city_lon']].copy()
hotel_df

,city,city_country,city_lat,city_lon
5,rikitea,PF,-23.12,-134.97
63,ovalle,CL,-30.60,-71.20
77,aswan,EG,24.09,32.91
118,agadez,NE,19.75,10.25
222,mecca,SA,21.43,39.83
223,hays,US,30.05,-98.03
234,salalah,OM,17.02,54.09
237,san carlos de bariloche,AR,-41.15,-71.31
264,khipro,PK,25.83,69.38
266,siguiri,GN,11.67,-9.50


In [11]:
hotel_df['Hotel Name'] = ' '

In [12]:
hotel_df.head()

,city,city_country,city_lat,city_lon,Hotel Name
5,rikitea,PF,-23.12,-134.97,
63,ovalle,CL,-30.60,-71.20,
77,aswan,EG,24.09,32.91,
118,agadez,NE,19.75,10.25,
222,mecca,SA,21.43,39.83,


In [13]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
# find the closest restaurant of each type to coordinates


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row['city_lat']
    lon = row['city_lon']

    # add keyword to params dict
    params['location'] = f'{lat},{lon}'

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

# Plot the hotels on top of the humidity heatmap with each pin containing the **Hotel Name**, **City**, and **Country**.
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,city,city_country,city_lat,city_lon,Hotel Name
5,rikitea,PF,-23.12,-134.97,Pension Maro'i
63,ovalle,CL,-30.60,-71.20,Hotel Altos de Tuqui
77,aswan,EG,24.09,32.91,Nuba Nile
118,agadez,NE,19.75,10.25,
222,mecca,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
223,hays,US,30.05,-98.03,Sinya on Lone Man Creek
234,salalah,OM,17.02,54.09,HAMDAN PLAZA HOTEL
237,san carlos de bariloche,AR,-41.15,-71.31,Hotel Patagonia
264,khipro,PK,25.83,69.38,Raja Hotel
266,siguiri,GN,11.67,-9.50,


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{city_country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["city_lat", "city_lon"]]

In [16]:
# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(markers)

fig
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))